In [1]:
import pandas as pd
from ggmodel_dev.models.utils import all_model_dictionary, all_model_properties_df


In [14]:
all_model_dictionary['EW1_partial_model']

In [4]:
'WP_rate_one'.rstrip('_one').rstrip('_two')

'WP_ra'

In [ ]:
'WP_rate_one'.rstrip('_one').rstrip('_two')

In [6]:
'WP_rate_one'[:-4]

'WP_rate'

In [9]:
value = 'Landuse'


[{'label': 'BE2_model', 'value': 'Change in Land Use'},
 {'label': 'BE3_model', 'value': 'Change in above group Biomass'},
 {'label': 'SL1_model', 'value': 'Nutrient Balance'},
 {'label': 'GE3_model', 'value': 'Agricultural Non-CO2 Emissions'}]